# Time series till it hurts

**Table of contents**
- [Introduction:Business Problem](#introduction)
- [Data](#data)
- [Methodology](#methodology)
- [Analysis](#analysis)
- [Results](#results)
- [Conclusion](#conclusion)
- [References](#references)

- Talk about the need for time series
- Difference between intapolation(regular regression) and 

extrapolation(time series)

## Introduction:Business Problem <a name="introduction"></a>

As with all forms of software devoloment, there is a multitute of ways to achive the same task. Frameworks are used to streamline the procces of software development. The issue is for a new developer(or Data science) in choosing what framework to begin their development journey.  

This project wiii use time series analysis to help show the popularity of different frameworks within the data science dormain

## Data <a name="data"></a>

We are using stack overflow data, on kaggle.

In [1]:
frameworks_to_forcast = ['nltk', 'stanford-nlp', 'python', 'r', 'numpy', 'scipy',
       'pandas', 'pytorch', 'keras', 'nlp',
       'hadoop', 'python-3.x', 'tensorflow',
      'lstm', 'seaborn',
      'plotly', 'scikit-learn', 'BeautifulSoup']

Make sure the Index colum of the data set is a **datetime index**

In [9]:
#pd.to_datetime(df.index)

We area going to have 3 models:
- ARIMA
- Auto ARIMA
- FbProphet

## Methodology <a name="methodology"></a>

1. user inputs a frame work 
2. user gets needs to choose method of evaluation
    - ARIMA
        Check stationarity(if True):
            - Show Adfuller Result
            - Plot the Correlation and AutoCorrelation Charts
            - Construct ARIMA model based on data(user can choose p,I,q.)
            - Graph Fitted model vs Original data
            - Use the model to make predictions(1 year) and plot predictions
        (If False):
            Choose auto ARIMA
    - auto ARIMA
        - automate best params and fit model
        - Show model residuals
        - Use the model to make predictions(1 year) and plot predictions
   
    - FbProphet
        - make appropriate DataFrame for prophet to make analysis
        - Construct model and Use the model to make Forecast(1 year)
        - plot forecast
        - plot_components() , shows trend and seasonality

## Analysis <a name="analysis"></a>

**Imports for the project**

In [2]:
# auto arima #
import pmdarima as pm

# ARIMA model #
from statsmodels.tsa.arima.model import ARIMA
# auto corrolation and partial auto corrolation function #
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Formal test for checking stationarity #
from statsmodels.tsa.stattools import adfuller

# matplotlib for plotting #
import matplotlib.pylab as matpy
matpy.rcParams['figure.figsize'] = (11, 8)
import matplotlib.pyplot as plt

# pandas and numpy for data manipulation #
import pandas as pd
import numpy as np

# for user interaction #
import streamlit as st

# Facebook Prophet  #
from prophet import Prophet
import warnings
warnings.simplefilter("ignore")

2022-04-28 09:27:43.863 ERROR   prophet.plot: Importing plotly failed. Interactive plots will not work.


**Functions used in the project** 

In [3]:
# Quick way to do addfuller test #
def quick_adfuller(data):
    # as a default we say the time series is stationary
    stationarity_value = True
    dftest = adfuller(data, autolag='AIC')

    st.write("1. ADF : ", dftest[0])
    st.write("2. P-Value : ", dftest[1])
    st.write("3. Num Of Lags : ", dftest[2])
    st.write(
        "4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
    st.write("5. Critical Values :")
    for key, val in dftest[4].items():
        st.write("\t", key, ": ", val)

    if dftest[1] <= 0.05:
        st.subheader("**Time series is stationary**")
        stationarity_value = True
    else:
        st.subheader("Time series is not stationary")
        stationarity_value = False

    return stationarity_value

# plotting auto corrolation function #
def acf_plot(data):
    # acf used for MA ie acf=q
    fig = plot_acf(data.values)
    plt.grid(True)
    plt.xticks(np.arange(17))
    st.pyplot(fig)

# plotting partial auto corrolation function #
def pacf_plot(data):
    # pacf used for AR ie pacf=p
    fig = plot_pacf(data.values)
    plt.grid(True)
    plt.xticks(np.arange(17))
    st.pyplot(fig)

    
# Plot forecast for auto corolation function #
def plot_arima_focast(train, test, forecast_series):
    fig = plt.figure(figsize=(12, 5), dpi=100)

    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(forecast_series, label='forecast')
    # plt.fill_between(lower_series.index, lower_series, upper_series,
    #                  color='k', alpha=.15)
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8)
    plt.grid()
    st.pyplot(fig)


# Facebook Prophet forecast plot #    
def fbProphet_plot(model_obj, forecast_df):
    fig = model_obj.plot(forecast_df)
    st.pyplot(fig)

# Facebook prophet shows tred and seasonality #
def fbProphet_plot_components(model_obj, forecast_df):
    fig = model_obj.plot_components(forecast_df)
    st.pyplot(fig)

    
# plot auto arima residuals #
def auto_ARIMA_risiduals(model_fitted):
    fig = model_fitted.plot_diagnostics(figsize=(12, 8))
    st.pyplot(fig)

# Plot residual errors ARIMA #
def plot_ARIMA_residuals(res):
    fig, ax = plt.subplots(1, 2, figsize=(11, 1.5))
    res.plot(title="Residuals", ax=ax[0])
    res.plot(kind="kde", title="Density", ax=ax[1])
    st.pyplot(fig)


In [4]:
## Loading Data #
#df = pd.read_csv("FrameworkData.csv", index_col=["month"])
#df.index = pd.to_datetime(df.index, yearfirst=True)

## User Chooses ARIMA model

**Check stationarity(if True):**

We use The addfuller test to check if the time series is stationary.[1.link](https://www.statology.org/dickey-fuller-test-python/)


If time series is stationarity, p value must be **equal to** or **less than 0.05**

- **Show Adfuller Result**


In [6]:
#stationarity = quick_adfuller(df[add_selectbox_framework])

- **Plot the Correlation and AutoCorrelation Charts**

In [8]:
acf_plot(df[add_selectbox_framework])
pacf_plot(df[add_selectbox_framework])

The structure of an ARIMA model: **ARIMA(p,I,q)**

We use **pacf** to obtain the **p value**. 

We use **acf** to obtain the **q value**.

Since the model is Sationary, **differencing=0(I=0)**.[2.link](https://predictivehacks.com/arima-model-in-python/)

- **Construct ARIMA model based on data(user can choose p,I,q.)**

In [10]:
# fitting arima model #
arima_model = ARIMA(df[add_selectbox_framework], order=(p_value, 0, q_value))
arima_fit = arima_model.fit()

# put the fitted values into a DataFrame to be displayed #
arima_df = pd.DataFrame(arima_fit.fittedvalues, index=df.index, columns=["fitted"])

# add the original values to the arima_fit_df dataframe #
arima_df["Original figures"] = df[add_selectbox_framework]

- **Graph Fitted model vs Original data**

In [11]:
st.subheader("Fitted and Original Graph")
st.line_chart(arima_df)

st means I am using the streamlit libriary.[3.link](https://docs.streamlit.io/library/api-reference)

- **Use the model to make predictions(1 year) and plot predictions and residuals**

In [12]:
# 1 YEAR PREDICTION (we have 36 data entries.12 per year) #
        train_val = df[add_selectbox_framework][:24]
        test_val = df[add_selectbox_framework][24:]

        with st.expander("Forecast"):
            # Build Model
            model = ARIMA(train_val, order=(p_value, 0, q_value))
            fitted = model.fit()

            # Residuals #
            residuals = pd.DataFrame(fitted.resid)

            # Forecast
            fc, se, conf = fitted.forecast(3, alpha=0.05)  # 95% conf

            # # Make as pandas series
            fc_series = pd.Series(fc, index=test_val.index)

            # partitioning dataframes #
            col_fc, col_res = st.columns(2)

            with col_fc:
                # display forcast as a series #
                st.subheader("Forecast values")
                st.dataframe(fc_series)
            with col_res:
                # show residuals values #
                st.subheader("Residual values")
                st.write(residuals)

            # show residuals plot #
            plot_ARIMA_residuals(residuals)

            # show forecast plot #
            plot_arima_focast(train_val, test_val, fc_series)

**(If Stationarity is False):**  Choose auto ARIMA model

## User Chooses auto ARIMA model

auto ARIMA is used in selecting the **best parameters** for the ARIMA model.
We are using auto Arima in this case because since the time series is not stationary, **I!=0**.
auto ARIMA accounts for the differencing factor.

- **automate best params and fit model**

In [14]:
# auto arima getting params #
auto_model = pm.auto_arima(df[add_selectbox_framework].values, start_p=1, start_q=1,
                           test='adf',       # use adftest to find optimal 'd'
                           max_p=4, max_q=4,  # maximum p and q
                           m=1,              # frequency of series
                           d=None,           # let model determine 'd'
                           seasonal=False,   # No Seasonality
                           start_P=0,
                           D=0,
                           trace=True,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True)

# display auto arima model summary #
with st.expander("Best model"):
    st.subheader("Auto ARIMA summary")
    st.write(auto_model.summary())

This is Similar to hyperparameter tunning in classification model.

Fiiting model

In [15]:
# train and test set #
train_auto_arima = df[add_selectbox_framework][:33]
test_auto_arima = df[add_selectbox_framework][33:]

# auto_model.fit(train_auto_arima)

- **Show model residuals**

In [17]:
# show risiduals #
with st.expander("Show model residuals"):
    st.subheader("Residuals")
    auto_ARIMA_risiduals(auto_model)

- **Use the model to make predictions(1 year) and plot predictions**

In [20]:
# model forecast #
with st.expander("forecast"):
    forecast_auto = auto_model.predict(len(df[add_selectbox_framework]))
    forecast_auto = pd.DataFrame(forecast_auto, index=df.index,
                                 columns=['Prediction'])

    # add the original data to the forecast data #
    forecast_auto["Original"] = df[add_selectbox_framework]

    col3, col4 = st.columns([2, 1])

    col3.subheader("Forecast Graph")
    col3.line_chart(forecast_auto)

    col4.subheader("Forecast DataFrame")
    col4.write(forecast_auto)

## User Chooses FbProphet model

(talk about the structure etc)

- **make appropriate DataFrame for prophet to make analysis**

(add picture) of example dataframe

- **Construct model and Use the model to make Forecast(1 year)**

In [21]:
# definning model #
m = Prophet()
model = m.fit(df_prophet)

# make predicitons for the next 13 months #
# 13 because our data starts at the begining of the month while fbProphet starts at the end of the month #
future = m.make_future_dataframe(periods=13, freq='M')
forecast = m.predict(future)

# button to show forecast data #
with st.expander("Show forecast"):
    st.subheader("Forecast DataFrame")
    st.write(forecast)

(add forecast dataframe)

- **plot forecast**

In [22]:
# button to plot forecast #
with st.expander("Plot forecast"):
    # plot forecast #
    st.subheader("Year Forecast")
    fbProphet_plot(m, forecast)

(add forecast plot)

- **plot_components() , shows trend and seasonality**

In [23]:
# button to plot Trend and Seasonality #
with st.expander("Trend and Seasonality"):
    # show trend and seasonality #
    st.subheader("Model Components")
    fbProphet_plot_components(m, forecast)

(show trend and seasonality plot)

## Results <a name="results"></a>

(add image of app)

## Conclusion <a name="conclusion"></a>

We can observe that from the arima models, when we forcast we regress towards the mean.

This is expected because arima doesnt count for seasonality within the time series and the data has too few samples.[4.link](https://stats.stackexchange.com/questions/333092/why-i-get-the-same-predict-value-in-arima-model)

An improvement would be to add a **SARIMA**(seasonal autoregressive integrated moving average)

The project could also show metric analysis and interactive plots using plotly

# References <a name="references"></a>

link3 -- https://facebook.github.io/prophet/docs/quick_start.html#python-api